In [107]:
# install tomato_disease_data_set from kaggel

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [3]:
# Data Augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

In [5]:
# Data Augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

In [7]:

# Set image size (InceptionV3 expects 299x299 images)
IMG_SIZE = (299, 299)
BATCH_SIZE = 32

# Define dataset paths
train_dir = "New Plant Diseases Dataset(Augmented)/train"
val_dir = "New Plant Diseases Dataset(Augmented)/valid"
# Load images from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

Found 18345 images belonging to 10 classes.
Found 4585 images belonging to 10 classes.


In [9]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
import numpy as np
from glob import glob
# Load InceptionV3 without the top layer (pre-trained model)
inception = InceptionV3(weights="imagenet", include_top=False, input_shape=(299, 299, 3))

# don't train existing weights
for layer in inception.layers:
    layer.trainable = False
  # useful for getting number of output classes
folders = glob('New Plant Diseases Dataset(Augmented)/train/*')
# our layers - you can add more if you want
x = Flatten()(inception.output)
prediction = Dense(len(folders), activation='softmax')(x)



# create a model object
model = Model(inputs=inception.input, outputs=prediction)
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 299, 299, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 149, 149, 32)      │             864 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 149, 149, 32)      │              96 │ conv2d[0][0]               │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation (Activation)       │ (None, 149, 149, 32)      │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (None, 147, 147, 32)      │           9,216 │ activation[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 147, 147, 32)      │              96 │ conv2d_1[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_1 (Activation)     │ (None, 147, 147, 32)      │               0 │ batch_normalization_1[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 147, 147, 64)      │          18,432 │ activation_1[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_2         │ (None, 147, 147, 64)      │             192 │ conv2d_2[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_2 (Activation)     │ (None, 147, 147, 64)      │               0 │ batch_normalization_2[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d (MaxPooling2D)  │ (None, 73, 73, 64)        │               0 │ activation_2[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (None, 73, 73, 80)        │           5,120 │ max_pooling2d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_3         │ (None, 73, 73, 80)        │             240 │ conv2d_3[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_3 (Activation)     │ (None, 73, 73, 80)        │               0 │ batch_normalization_3[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_4 (Conv2D)             │ (None, 71, 71, 192)       │         138,240 │ activation_3[0][0]         │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 23,113,514 (88.17 MB)

 Trainable params: 1,310,730 (5.00 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [11]:

# Train the model
EPOCHS = 10
history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=EPOCHS)

# Save the trained model
model.save("tomato_disease_model.h5")

print("Model training complete and saved as 'tomato_disease_model.h5'")

C:\Users\munap\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
574/574 ━━━━━━━━━━━━━━━━━━━━ 2887s 5s/step - accuracy: 0.5974 - loss: 1.4103 - val_accuracy: 0.7538 - val_loss: 0.8197
Epoch 2/10
574/574 ━━━━━━━━━━━━━━━━━━━━ 2803s 5s/step - accuracy: 0.7932 - loss: 0.6932 - val_accuracy: 0.8408 - val_loss: 0.5229
Epoch 3/10
574/574 ━━━━━━━━━━━━━━━━━━━━ 2974s 5s/step - accuracy: 0.8313 - loss: 0.5947 - val_accuracy: 0.8475 - val_loss: 0.5259
Epoch 4/10
574/574 ━━━━━━━━━━━━━━━━━━━━ 3001s 5s/step - accuracy: 0.8325 - loss: 0.6262 - val_accuracy: 0.8388 - val_loss: 0.6112
Epoch 5/10
574/574 ━━━━━━━━━━━━━━━━━━━━ 2922s 5s/step - accuracy: 0.8444 - loss: 0.6025 - val_accuracy: 0.8663 - val_loss: 0.4973
Epoch 6/10
574/574 ━━━━━━━━━━━━━━━━━━━━ 2125s 4s/step - accuracy: 0.8632 - loss: 0.5406 - val_accuracy: 0.8484 - val_loss: 0.5791
Epoch 7/10
574/574 ━━━━━━━━━━━━━━━━━━━━ 1766s 3s/step - accuracy: 0.8681 - loss: 0.5384 - val_accuracy: 0.8395 - val_loss: 0.6751
Epoch 8/10
574/574 ━━━━━━━━━━━━━━━━━━━━ 1830s 3s/step - accuracy: 0.8684 - loss: 0.5384 - 

Model training complete and saved as 'tomato_disease_model.h5'


In [103]:
import cv2
import numpy as np
img_path="img.JPG"
def preprocess_image(image_path):
    img = cv2.imread(image_path)  # Load image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB (if using OpenCV)
    img = cv2.resize(img, (299, 299))  # Resize to InceptionV3 input size
    img = img / 255.0  # Normalize pixel values (0 to 1)
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img
image = preprocess_image(img_path)
prediction = model.predict(image)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 556ms/step


array([[4.6110004e-02, 6.2710285e-01, 4.0106248e-02, 5.4495744e-03,
        6.9979724e-04, 4.4055088e-04, 2.8009096e-01, 6.0831937e-08,
        1.2616083e-11, 3.6084380e-08]], dtype=float32)

In [104]:
class_labels = [
    "Bacterial Spot",
    "Early Blight", 
    "Healthy",  # Class 0
    "Late Blight",  # Class 1
    "Leaf Mold", 
    "Septoria Leaf Spot",  
    "Spider Mites"
    "Target Spot",  
    "Mosaic Virus",  
    "Yellow Leaf Curl Virus",  
]

# Get the index of the highest probability
predicted_class = np.argmax(prediction)
# Map the predicted class to the label
predicted_label = class_labels[predicted_class]

print(f"Predicted Class Index: {predicted_class}")
print(f"Predicted Disease: {predicted_label}")

Predicted Class Index: 1
Predicted Disease: Early Blight
